In [ ]:
# Import useful libraries.
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
import scipy.io
from scipy import stats
import math
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance
from sklearn.model_selection import RandomizedSearchCV
import matplotlib
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from scipy.spatial import distance
from sklearn.ensemble import IsolationForest
from scipy.stats import mode

## Data Preprocessing

### Load data

In [ ]:
# load directory of data.
direct = ''
mat = scipy.io.loadmat(direct) 

### Data split

In [ ]:
# Split dataset (allspec) into training set and test set.
X_train, X_test, y_train, y_test = train_test_split(data_all, # define the spectrum data
                                                    np.ravel(label_number), # label of spectrum data drug MoAs
                                                    test_size = 0.3, 
                                                    stratify = np.ravel(label_number), 
                                                    random_state=109) # 70% training and 30% test

## Model construction

### LDA model

In [ ]:
lda_clf = LinearDiscriminantAnalysis(priors = prior, 
                                     store_covariance = True).fit(X_train,y_train)

### Model evalutaion

In [ ]:
# Evaluate model performance using accuracy.
lda_pred = lda_clf.predict(X_test)
lda_pred_accu = accuracy_score(y_test, lda_pred)
print("Accuracy lda for allspec:", lda_pred_accu)

In [ ]:
# Plot confusion matrix.
cm = confusion_matrix(y_test, lda_pred, normalize='true')
# label of each drug MoA.
name_list = ['Control','PSI','PDI','DNAI/TopoII','LFACS','FASN','mTOR/PI3K','EGFR/HER2','EGFR','PARP']

df_cm = pd.DataFrame(cm, index = [i for i in name_list],
                     columns = [i for i in name_list])
plt.figure(figsize = (10,6))
plt.rcParams.update({'font.size':10})
plt.title('Confusion matrix of LDA classifier')
sns.heatmap(df_cm, annot=True, fmt='.2%')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()

## Novelty detection

In [ ]:
# Perform PCA reduction to keep top 50 principal components in novelty detection, 
# we found this step will generate more reasonable visualization results in LDA plots when there is new data from test compound,
# as the total of 288 dimmensions is hard to be kept in only 3 dimmensions. Otherwise, the distribution of test compound will
# be very dispersed. This step also helps in predict the closest group.
pca = PCA(n_components=50,svd_solver='full')
data_mean = np.mean(data_all,axis = 0)
data_all_reduced = pca.fit_transform(data_all - data_mean) # centralize data before PCA.

newspec = cellspec_epi # Use epirubicin spec as an example of the tested compound.
newspec_reduced = pca.transform(newspec - data_mean) # Transform spec of tested compound to PCA reduction space.

In [ ]:
# For novelty detection, we can construct a new LDA model using all the data.
lda_clf_novel = LinearDiscriminantAnalysis(priors = prior, 
                                           store_covariance = True).fit(data_all_reduced,np.ravel(label_number))

In [ ]:
# A simple way to calcualte the closest reference group is to directly use the predict function in LDA, 
# which intrinsically calculated the Mahalanobis distance.
idx_matrix = lda_clf_novel.predict(newspec_reduced)
idx = mode(idx_matrix)[0]
print(idx)

In [205]:
# To obtain the specific/quantative Mahalanobis distance, reference the source code at:
# https://github.com/scikit-learn/scikit-learn/blob/2d8e03f4d/sklearn/discriminant_analysis.py#L177
def empirical_covariance(X, *, assume_centered=False):
    """Compute the Maximum likelihood covariance estimator.
    Parameters
    ----------
    X : ndarray of shape (n_samples, n_features)
        Data from which to compute the covariance estimate.
    assume_centered : bool, default=False
        If `True`, data will not be centered before computation.
        Useful when working with data whose mean is almost, but not exactly
        zero.
        If `False`, data will be centered before computation.
    Returns
    -------
    covariance : ndarray of shape (n_features, n_features)
        Empirical covariance (Maximum Likelihood Estimator).
    """
    X = np.asarray(X)

    if X.ndim == 1:
        X = np.reshape(X, (1, -1))

    if X.shape[0] == 1:
        warnings.warn(
            "Only one sample available. You may want to reshape your data array"
        )

    if assume_centered:
        covariance = np.dot(X.T, X) / X.shape[0]
    else:
        covariance = np.cov(X.T, bias=1)

    if covariance.ndim == 0:
        covariance = np.array([[covariance]])
    return covariance


def _cov(X, shrinkage=None, covariance_estimator=None):
    """Estimate covariance matrix (using optional covariance_estimator).
    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)
        Input data.
    shrinkage : {'empirical', 'auto'} or float, default=None
        Shrinkage parameter, possible values:
          - None or 'empirical': no shrinkage (default).
          - 'auto': automatic shrinkage using the Ledoit-Wolf lemma.
          - float between 0 and 1: fixed shrinkage parameter.
        Shrinkage parameter is ignored if  `covariance_estimator`
        is not None.
    covariance_estimator : estimator, default=None
        If not None, `covariance_estimator` is used to estimate
        the covariance matrices instead of relying on the empirical
        covariance estimator (with potential shrinkage).
        The object should have a fit method and a ``covariance_`` attribute
        like the estimators in :mod:`sklearn.covariance``.
        if None the shrinkage parameter drives the estimate.
        .. versionadded:: 0.24
    Returns
    -------
    s : ndarray of shape (n_features, n_features)
        Estimated covariance matrix.
    """
    if covariance_estimator is None:
        shrinkage = "empirical" if shrinkage is None else shrinkage
        if isinstance(shrinkage, str):
            if shrinkage == "auto":
                sc = StandardScaler()  # standardize features
                X = sc.fit_transform(X)
                s = ledoit_wolf(X)[0]
                # rescale
                s = sc.scale_[:, np.newaxis] * s * sc.scale_[np.newaxis, :]
            elif shrinkage == "empirical":
                s = empirical_covariance(X)
        elif isinstance(shrinkage, Real):
            s = shrunk_covariance(empirical_covariance(X), shrinkage)
    else:
        if shrinkage is not None and shrinkage != 0:
            raise ValueError(
                "covariance_estimator and shrinkage parameters "
                "are not None. Only one of the two can be set."
            )
        covariance_estimator.fit(X)
        if not hasattr(covariance_estimator, "covariance_"):
            raise ValueError(
                "%s does not have a covariance_ attribute"
                % covariance_estimator.__class__.__name__
            )
        s = covariance_estimator.covariance_
    return s

def _class_cov(X, y, shrinkage=None, covariance_estimator=None):
    """Compute weighted within-class covariance matrix.
    The per-class covariance are weighted by the class priors.
    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)
        Input data.
    y : array-like of shape (n_samples,) or (n_samples, n_targets)
        Target values.
    priors : array-like of shape (n_classes,)
        Class priors.
    shrinkage : 'auto' or float, default=None
        Shrinkage parameter, possible values:
          - None: no shrinkage (default).
          - 'auto': automatic shrinkage using the Ledoit-Wolf lemma.
          - float between 0 and 1: fixed shrinkage parameter.
        Shrinkage parameter is ignored if `covariance_estimator` is not None.
    covariance_estimator : estimator, default=None
        If not None, `covariance_estimator` is used to estimate
        the covariance matrices instead of relying the empirical
        covariance estimator (with potential shrinkage).
        The object should have a fit method and a ``covariance_`` attribute
        like the estimators in sklearn.covariance.
        If None, the shrinkage parameter drives the estimate.
        .. versionadded:: 0.24
    Returns
    -------
    cov : array-like of shape (n_features, n_features)
        Weighted within-class covariance matrix
    """
    classes = np.unique(y)
    cov = np.zeros(shape=(X.shape[1], X.shape[1]))
    priors = np.bincount(y) / float(len(y))
    for idx, group in enumerate(classes):
        Xg = X[y == group]
        cov += priors[idx] * np.atleast_2d(_cov(Xg, shrinkage, covariance_estimator))
    return cov

def get_class(X, y, cov_X, number):
    """
    Compute the mean of each class and the inverse of covariance of each class
    Parameters
    -------
    X: data
    Y: label
    cov_X: covariance of X, can be computed by _class_cov function
    number: number of classes
    Returns
    -------
    class_mean_list : centroid of each class
    covinv_list: inverse of covariance of each class
    """
    class_mean_list = []
    covinv_list = []
    for i in range(number):
        cl = X[y == i]
        cov = cov_X
        inverse=np.linalg.pinv(cov)
        cl_mean = np.mean(cl,axis=0)
        class_list.append(cl)
        class_mean_list.append(cl_mean)
        covinv_list.append(inverse)
    return class_mean_list, covinv_list

def get_mahalanobis_dist(class_mean_list, convinv_list, spec):
    """
    Compute the Mahalanobis distance between the input spec data and each class
    Parameters
    -----------
    class_mean_list: centroid of each class
    covinv_list: inverse of covariance of each class
    spec: spec of tested compound
    Returns
    -------
    dist_matrix : Mahalanobis distance of the sepc data from tested compound compared with each class centroid
    """
    dist_matrix = np.zeros((len(spec),len(class_mean_list)))
    for i in range(len(spec)):
        for j in range(len(class_mean_list)):
            dist = distance.mahalanobis(spec[i,:], class_mean_list[j], convinv_list[j])
            dist_matrix[i,j] = dist
    return dist_matrix

In [ ]:
cov_X = _class_cov(data_all_reduced, np.ravel(label_number)) # Compute covariance of each class

class_mean_list, covinv_list = get_class(data_all_reduced, 
                                         np.ravel(label_number), cov_X, 10) # Compute centroid and inverse of covrariance matrix

dist_m = get_mahalanobis_dist(class_mean_list,
                              covinv_list,
                              newspec_reduced) # Compute Mahalanobis distance of tested spec

In [ ]:
# Calculate the average novelty prediction score using isolation forest 
# to detect whether the tested compound is outlier compared to its closest reference group.
nov_list = []
clf = IsolationForest(contamination = 0.2, 
                      random_state= 10,
                      n_estimators = 1000).fit(data_all[np.ravel(label_number) == idx])

nov = clf.predict(newspec)
print("novelty score: ", np.mean(nov))
print("outlier percentage:", len(nov[nov == -1])/len(nov))